In [ ]:
!pip uninstall -y wandb
!pip uninstall -y torch torchvision torchaudio ultralytics -q
!pip install -q torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0 --index-url https://download.pytorch.org/whl/cpu
!pip install -q ultralytics==8.1.13 tqdm kaggle

ERROR: Operation cancelled by user
^C


In [2]:
import os
os.environ['WANDB_DISABLED'] = 'true'

In [3]:
from pathlib import Path

Path('/content/helmetbehincode').mkdir(parents=True, exist_ok=True)

data_yaml_text = """
path: /content/helmetbehincode
train: train/images
val: valid/images

names:
  0: rider
  1: helmet
  2: no_helmet
"""

Path('/content/helmetbehincode/data.yaml').write_text(data_yaml_text)
print("✅ Novo data.yaml criado com as 4 classes.")

✅ Novo data.yaml criado com as 4 classes.


In [4]:
from google.colab import files
import os

if not os.path.exists('/root/.kaggle/kaggle.json'):
    files.upload()
    !mkdir -p ~/.kaggle && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d abuzarkhaaan/helmetbehncode -p /content --unzip

Dataset URL: https://www.kaggle.com/datasets/abuzarkhaaan/helmetbehncode
License(s): Community Data License Agreement - Permissive - Version 1.0
 54% 69.0M/128M [00:00<00:00, 719MB/s]
100% 128M/128M [00:00<00:00, 486MB/s] 


In [5]:
import shutil

base_dir = '/content'
dest_dir = os.path.join(base_dir, 'helmetbehincode')
os.makedirs(dest_dir, exist_ok=True)

for folder in ['train', 'valid', 'test']:
    src = os.path.join(base_dir, folder)
    dst = os.path.join(dest_dir, folder)
    if os.path.exists(src):
        shutil.move(src, dst)
        print(f"✅ Movido: {folder}/ → helmetbehincode/{folder}/")
    else:
        print(f"⚠️ Pasta não encontrada: {src}")

✅ Movido: train/ → helmetbehincode/train/
✅ Movido: valid/ → helmetbehincode/valid/
✅ Movido: test/ → helmetbehincode/test/


In [ ]:
from ultralytics import YOLO
import random
import glob
import os
from IPython.display import Image, display

model = YOLO('yolov8n.pt')

results = model.train(
    data='/content/helmetbehincode/data.yaml',
    epochs=5,
    imgsz=640,
    batch=16,
    name='helmet_detection_extended4',
    project='helmet-detection'
)

modelo_treinado = YOLO('/content/helmet-detection/helmet_detection_extended4/weights/best.pt')

image_paths = glob.glob('/content/helmetbehincode/valid/images/*.jpg')

for i in range(3):
    img_path = random.choice(image_paths)

    pred = modelo_treinado.predict(source=img_path, conf=0.25, save=True, verbose=False)
    pred_path = os.path.join(pred[0].save_dir, os.path.basename(img_path))

    display(Image(filename=pred_path))
    print(f"🖼️ Exibind imagens com prediçãos: {os.path.basename(img_path)}")

New https://pypi.org/project/ultralytics/8.3.161 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.13 🚀 Python-3.11.13 torch-2.3.0+cpu CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/helmetbehincode/data.yaml, epochs=5, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=helmet-detection, name=helmet_detection_extended45, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fa

train: Scanning /content/helmetbehincode/train/labels.cache... 1629 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1629/1629 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



/usr/local/lib/python3.11/dist-packages/ultralytics/data/augment.py:846: UserWarning: Argument(s) 'quality_lower' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=75, p=0.0),
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()
val: Scanning /content/helmetbehincode/valid/labels.cache... 155 images, 0 backgrounds, 0 corrupt: 100%|██████████| 155/155 [00:00<?, ?it/s]

Plotting labels to helmet-detection/helmet_detection_extended45/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to helmet-detection/helmet_detection_extended45
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/5         0G      1.537      2.422      1.872         74        640:  51%|█████     | 52/102 [10:59<10:34, 12.70s/it]

In [1]:
metrics = model.val(data='/content/helmetbehincode/data.yaml')
print(f"mAP50-95: {metrics.box.map:.4f}")
metrics.confusion_matrix.plot(save_dir='.')

NameError: name 'model' is not defined

In [ ]:
!find /content -name "best.pt"

In [ ]:
from google.colab import files
files.download('/content/helmet-detection/helmet_detection_extended2/weights/best.pt')

In [ ]:
import random
import glob
import os
from IPython.display import Image, display
from ultralytics import YOLO

model = YOLO('/content/helmet-detection/helmet_detection_extended2/weights/best.pt')

image_paths = glob.glob('/content/helmetbehincode/valid/images/*.jpg')

for i in range(3):
    random_image = random.choice(image_paths)

    results = model.predict(
        source=random_image,
        conf=0.25,
        save=True
    )

    pred_path = os.path.join(results[0].save_dir, os.path.basename(random_image))

    display(Image(filename=pred_path))
    print(f"✅ ({i+1}/3) Imagem: {os.path.basename(random_image)}") #caso, professor, deseja aumentar o nº de imagens, altere o "/3" para o nº q desejar!!!

In [ ]:
from google.colab import files
from ultralytics import YOLO
from IPython.display import Image, display
import os

uploaded = files.upload()
fname = next(iter(uploaded))

model = YOLO('/content/helmet-detection/helmet_detection_extended2/weights/best.pt')

results = model.predict(
    source=fname,
    conf=0.25,
    save=True
)

save_dir = results[0].save_dir
pred_path = os.path.join(save_dir, fname)

display(Image(pred_path))
print("✅ Detecção finalizada e imagem exibida!")

In [ ]:
# 1. Instala a biblioteca necessária para conversão
!pip install -q moviepy

# 2. Converte o GIF para MP4
import moviepy.editor as mp

gif_clip = mp.VideoFileClip("/content/teste.gif")  # Altere se o nome for diferente
gif_clip.write_videofile("/content/teste.mp4")


In [ ]:
from ultralytics import YOLO
model = YOLO('/content/helmet-detection/helmet_detection_extended2/weights/best.pt')

In [ ]:
results = model.predict(
    source="/content/teste.mp4",
    conf=0.1,
    save=True
)

In [ ]:
!ffmpeg -i /content/runs/detect/predict39/teste.avi /content/runs/detect/predict39/teste.mp4

In [ ]:
from IPython.display import Video
Video("/content/runs/detect/predict39/teste.mp4", embed=True)